In [1]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Polygon

In [18]:
df = pd.read_csv("taxable_values.csv",dtype=str)
df = df[0:37]
df['TAXING AUTHORITY'] = df['TAXING AUTHORITY'].str.replace('\d+','',regex=True)

In [19]:
gdf = gpd.read_file('Municipalitypoly_gdb_-3974701052066827405.geojson')

In [20]:
df['TAXING AUTHORITY'] = df['TAXING AUTHORITY'].str.strip()

In [21]:
gdf.columns

Index(['OBJECTID', 'MUNICUID', 'MUNICID', 'NAME', 'FIPSCODE', 'CREATEDBY',
       'CREATEDDATE', 'MODIFIEDBY', 'MODIFIEDDATE', 'GlobalID', 'geometry'],
      dtype='object')

In [22]:
gdf['NAME'] = gdf['NAME'].str.replace("UNINCORPORATED MIAMI-DADE","UNINCORPORATED")

In [23]:
gdf[gdf['NAME']

,OBJECTID,MUNICUID,MUNICID,NAME,FIPSCODE,CREATEDBY,CREATEDDATE,MODIFIEDBY,MODIFIEDDATE,GlobalID,geometry
0,1,74,24,KEY BISCAYNE,1236300.0,None,None,None,None,70964e2b-4266-4df7-b51c-a17364fff5f9,"POLYGON ((-80.16156 25.7004, -80.16139 25.7004..."
1,2,19,12,BAL HARBOUR,1203275.0,None,None,None,None,3e619e37-3c0d-4a67-adaa-ebaf892c1c1f,"POLYGON ((-80.12092 25.89954, -80.12092 25.899..."
2,3,66,09,SOUTH MIAMI,1267550.0,None,None,None,None,fbc529d2-2db4-46eb-aa35-dfce5b83c2e9,"POLYGON ((-80.28621 25.72626, -80.28615 25.724..."
3,4,71,09,SOUTH MIAMI,1267550.0,None,None,None,None,e367640b-25fc-4fd1-9df8-f8448d0d2930,"POLYGON ((-80.29229 25.72487, -80.29176 25.724..."
4,5,39,22,MEDLEY,1243900.0,None,None,E316899,"Thu, 22 Jun 2023 10:19:41 GMT",357fb0b5-2652-4ab3-959f-d94a7c3f9da7,"POLYGON ((-80.36379 25.88423, -80.35548 25.878..."
...,...,...,...,...,...,...,...,...,...,...,...
71,76,13,04,HIALEAH,1230000.0,None,None,None,None,8145221d-a929-4ea4-8446-7a4065dab69d,"POLYGON ((-80.34713 25.90026, -80.34675 25.899..."
72,77,44,30,UNINCORPORATED,NaN,None,None,None,None,985a285c-df88-49bb-90d5-3c2985ec5ca1,"POLYGON ((-80.26098 25.75807, -80.26095 25.757..."
73,78,35,35,DORAL,1217935.0,None,None,E316899,"Thu, 22 Jun 2023 10:17:25 GMT",dd966362-75a1-4e39-ae1e-faeb196f814c,"POLYGON ((-80.35471 25.85431, -80.35468 25.853..."
74,80,4,30,UNINCORPORATED,NaN,E316899,"Thu, 21 Apr 2022 16:34:08 GMT",E316899,"Thu, 21 Apr 2022 17:23:35 GMT",6d57fb7a-ea31-4b13-9e05-3811c3de948f,"POLYGON ((-80.14733 25.95905, -80.14724 25.956..."


In [7]:
gdf = gdf.dissolve(by="NAME",as_index=False)

In [8]:
merged_gdf = pd.merge(left=df,left_on='TAXING AUTHORITY',right=gdf,right_on='NAME',how='left')

In [9]:
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.title()
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.replace('Mb','MB')
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.replace('Dda','DDA')

In [10]:
empty_poly = Polygon()
merged_gdf["geometry"] = merged_gdf["geometry"].apply(
    lambda geom: geom if geom is not None else empty_poly
)

In [11]:
def compute_centroid(geom):
    if geom is None or geom.is_empty:
        return None
    return geom.centroid

merged_gdf["centroid"] = merged_gdf["geometry"].apply(compute_centroid)

# 3) (Optional) If you want separate latitude/longitude columns:
merged_gdf["centroid_x"] = merged_gdf["centroid"].apply(lambda pt: pt.x if pt is not None else None)
merged_gdf["centroid_y"] = merged_gdf["centroid"].apply(lambda pt: pt.y if pt is not None else None)

In [12]:
drop_columns = ['OBJECTID', 'MUNICUID',
       'MUNICID', 'NAME', 'FIPSCODE', 'CREATEDBY', 'CREATEDDATE', 'MODIFIEDBY',
       'MODIFIEDDATE', 'GlobalID']

In [13]:
merged_gdf = merged_gdf.drop(columns=drop_columns)

In [14]:
merged_gdf = gpd.GeoDataFrame(
    merged_gdf,
    geometry="geometry",
    crs="EPSG:4326"    
)



In [15]:
merged_gdf = merged_gdf.drop_duplicates(subset='TAXING AUTHORITY')

In [16]:
merged_gdf.to_file("merged_taxable_values.geojson", driver="GeoJSON")

In [17]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miamidade_taxable_values
